In [ ]:
#Test unravel and also upsampling method!
import tensorflow as tf
import numpy as np


def max_pool(inputs,name):
    value,index = tf.nn.max_pool_with_argmax(inputs,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name=name,
                                            Targmax = tf.int32)
    print('value shape',value.shape)
    print('index shape',index.shape)
    
    return value,index



def unravel_index(indices, shape):
    with tf.name_scope('unravel_index'):
        indices = tf.expand_dims(indices, 0)
        shape = tf.expand_dims(shape, 1)
        strides = tf.cumprod(shape, reverse=True)
        strides_shifted = tf.cumprod(shape, exclusive=True, reverse=True)
        return (indices % strides) // strides_shifted
    #This function is utilized to transform the flattened maxpooling index to the original 4D tensor, and have already test
    #it, which works brilliant!
    """
    indices: indices will be the output index from maxpooling, just to make sure if it's only 1D!
    Shape: the shape of the original data,[batch_size,height,width,Num_of_Channels]
    output: It's the 4D maxpooling indices!
    """
    
def up_sampling(value,indices,shape):
    """
    Inputs:
    value: the maximum value from maxpooling function, value need to be a tensor. The most important thing for
    value is that it needs to be reshaped to be only one column! 
    indices: the flattened position for the maximum value from maxpooling function. Also indices need to be reshaped
    to be two dimension. [Num_tot_values,4]. 4 is because we have 4 dimension
    shape: the shape of the original data, [batch_size,height,width,Num_of_Channels]
    Outputs:
    up_sample_sp: The sparse matrix from the up_sampling.
    """
    mx_sh = value.shape
    tot = mx_sh[0].value*mx_sh[1].value*mx_sh[2].value*mx_sh[3].value
    value_reshape = tf.reshape(value,[tot])
    index_reshape = tf.reshape(indices,[1,tot])
    print('The shape of reshaped maxvalue',value_reshape.shape)
    print('The shape of reshaped maxindex',index_reshape.shape)
    pooling_index_4d = tf.to_int64(unravel_index(index_reshape,shape),name='ToInt64')
    print('The shape of 4d indices', pooling_index_4d.shape)
    s = tf.Session()
    sp_tensor = tf.SparseTensor(tf.reshape(pooling_index_4d,
                                           [pooling_index_4d.shape[2].value,
                                            pooling_index_4d.shape[1].value]),
                                            values = value_reshape, dense_shape = shape)
    sp_dense = tf.sparse_tensor_to_dense(sp_tensor)
    print('The sp_dense?',s.run(sp_dense))
    print('The shape of sparse matrix', sp_dense.shape)
    return s.run(sp_dense) 


In [ ]:
# let's assume that inputs is a matrix which is 4D matrix, the dimension is (Num_Batch, width, height, Num_channel)
shape = [1,5,5,2]
temp = np.random.randint(20,size=shape)
max_value, max_index = max_pool(temp,name='PLAY')
value = max_value
indices = max_index


In [ ]:
mx_sh = value.shape
tot = mx_sh[0].value*mx_sh[1].value*mx_sh[2].value*mx_sh[3].value
value_reshape = tf.reshape(value,[tot])
index_reshape = tf.reshape(indices,[1,tot])
print('The shape of reshaped maxvalue',value_reshape.shape)
print('The shape of reshaped maxindex',index_reshape.shape)
pooling_index_4d = tf.to_int64(unravel_index(index_reshape,shape),name='ToInt64')
print('The shape of 4d indices', pooling_index_4d.shape)
s = tf.Session()
sp_tensor = tf.SparseTensor(tf.reshape(pooling_index_4d,
                                       [pooling_index_4d.shape[2].value,
                                        pooling_index_4d.shape[1].value]),
                            values = value_reshape, dense_shape = shape)
sp_dense = tf.sparse_tensor_to_dense(sp_tensor)

In [ ]:
s.run(sp_dense)